In [1]:
from morpheus.modflow.types.ModflowModel import ModflowModel, ModelId
from morpheus.modflow.types.Settings import Metadata, Name, Description, Tags
from morpheus.modflow.types.User import UserId
from morpheus.modflow.types.Project import Project, ProjectId
from morpheus.modflow.domain.events.ProjectCreatedEvent import ProjectCreatedEvent

user_id = UserId.new()
project_id = ProjectId.new()

project = Project.new(user_id=user_id, project_id=project_id)
settings = project.settings
project = project.with_updated_settings(
  settings.with_updated_metadata(Metadata(name=Name.from_str('Test'), description=Description.from_str('Test project'), tags=Tags.from_list([])))
)

project = project.with_updated_base_model(base_model=ModflowModel.new(
  model_id=ModelId.new()
))

event = ProjectCreatedEvent.from_project(project=project)

event

ProjectCreatedEvent(entity_uuid=Uuid(value='6e7bd89f-7783-4b88-aa46-83ca45c71b44'), payload={'project_id': '6e7bd89f-7783-4b88-aa46-83ca45c71b44', 'base_model': {'model_id': '9d2e9e3b-4dce-42c4-9836-6778019ab168', 'version': {'last_version': 'v0.0.0', 'current_version': 'v0.0.0-0', 'is_head': True}, 'spatial_discretization': {'geometry': {'type': 'Polygon', 'coordinates': [[(0, 0), (1, 0), (1, 1), (0, 1), (0, 0)]]}, 'affected_cells': None, 'grid': {'x_distances': [111319.49079327357], 'y_distances': [111325.1428663851], 'origin': {'type': 'Point', 'coordinates': (0.0, 0.9999999999999998)}, 'rotation': 0, 'length_unit': 'meters'}, 'crs': 'EPSG:4326'}, 'time_discretization': {'start_datetime': '2020-01-01T00:00:00+00:00', 'end_datetime': '2020-01-02T00:00:00+00:00', 'stress_periods': [{'start_datetime': '2020-01-01T00:00:00+00:00', 'number_of_time_steps': 1, 'time_step_multiplier': 1.0, 'steady_state': True}], 'time_unit': 4}, 'boundaries': [], 'observations': [], 'soil_model': [{'id': '

In [2]:
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.common.types.event_sourcing.EventEnvelope import EventEnvelope, OccurredAt

envelope = EventEnvelope(event, OccurredAt.now(), EventMetadata.from_dict({'test': 123}))
envelope

EventEnvelope(event=ProjectCreatedEvent(entity_uuid=Uuid(value='6e7bd89f-7783-4b88-aa46-83ca45c71b44'), payload={'project_id': '6e7bd89f-7783-4b88-aa46-83ca45c71b44', 'base_model': {'model_id': '9d2e9e3b-4dce-42c4-9836-6778019ab168', 'version': {'last_version': 'v0.0.0', 'current_version': 'v0.0.0-0', 'is_head': True}, 'spatial_discretization': {'geometry': {'type': 'Polygon', 'coordinates': [[(0, 0), (1, 0), (1, 1), (0, 1), (0, 0)]]}, 'affected_cells': None, 'grid': {'x_distances': [111319.49079327357], 'y_distances': [111325.1428663851], 'origin': {'type': 'Point', 'coordinates': (0.0, 0.9999999999999998)}, 'rotation': 0, 'length_unit': 'meters'}, 'crs': 'EPSG:4326'}, 'time_discretization': {'start_datetime': '2020-01-01T00:00:00+00:00', 'end_datetime': '2020-01-02T00:00:00+00:00', 'stress_periods': [{'start_datetime': '2020-01-01T00:00:00+00:00', 'number_of_time_steps': 1, 'time_step_multiplier': 1.0, 'steady_state': True}], 'time_unit': 4}, 'boundaries': [], 'observations': [], 'so

# Listeners must have arguments "event" and "metadata" with proper types

In [3]:
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to


class Invalid:
  pass


class ProjectorInvalid0:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self):
    print('Created project')


class ProjectorInvalid1:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid2:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
    print('Created project')


class ProjectorInvalid3:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: Invalid, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid4:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
    print('Created project')


event_publisher = EventPublisher()
try:
  projector = ProjectorInvalid0()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid1()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid2()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid3()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid4()
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x179978e00>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x179978ec0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x1795d5310>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x179978ec0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x1795d5310>> must have an arg

# Cannot register same listener twice

In [4]:
class Projector:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
    print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
  event_publisher = EventPublisher()
  projector = Projector()
  event_publisher.register(projector)
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event handler already registered


In [5]:
from morpheus.modflow.infrastructure.event_sourcing.ModflowEventBus import modflow_event_bus

modflow_event_bus.record(envelope) 

In [6]:
envelope

EventEnvelope(event=ProjectCreatedEvent(entity_uuid=Uuid(value='6e7bd89f-7783-4b88-aa46-83ca45c71b44'), payload={'project_id': '6e7bd89f-7783-4b88-aa46-83ca45c71b44', 'base_model': {'model_id': '9d2e9e3b-4dce-42c4-9836-6778019ab168', 'version': {'last_version': 'v0.0.0', 'current_version': 'v0.0.0-0', 'is_head': True}, 'spatial_discretization': {'geometry': {'type': 'Polygon', 'coordinates': [[(0, 0), (1, 0), (1, 1), (0, 1), (0, 0)]]}, 'affected_cells': None, 'grid': {'x_distances': [111319.49079327357], 'y_distances': [111325.1428663851], 'origin': {'type': 'Point', 'coordinates': (0.0, 0.9999999999999998)}, 'rotation': 0, 'length_unit': 'meters'}, 'crs': 'EPSG:4326'}, 'time_discretization': {'start_datetime': '2020-01-01T00:00:00+00:00', 'end_datetime': '2020-01-02T00:00:00+00:00', 'stress_periods': [{'start_datetime': '2020-01-01T00:00:00+00:00', 'number_of_time_steps': 1, 'time_step_multiplier': 1.0, 'steady_state': True}], 'time_unit': 4}, 'boundaries': [], 'observations': [], 'so